## just have a look at the case that without downsampling

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [5]:
df_train = pd.read_csv('data/train_clean.csv')
df_test = pd.read_csv('test.csv')
df_val = pd.read_csv('data/val_clean.csv')

train_x = df_train.drop(['click','bidid','userid','IP','city','domain', 'url','urlid','slotid','creative','bidprice','payprice','keypage'], axis=1)
train_y = df_train.click

val_x = df_val.drop(['click','bidid','userid','IP','city','domain', 'url','urlid','slotid','creative','bidprice','payprice','keypage'], axis=1)
val_y = df_val.click

test_x = df_test.drop(['bidid','userid','IP','city','domain', 'url','urlid','slotid','creative','keypage'], axis=1)

## Data preprocessing with one-hot

In [6]:
def pip(dataframe):
    #weekday
    dataframe = pd.concat([dataframe,pd.get_dummies(dataframe.weekday,prefix='day')],axis=1)
    #hour
    dataframe = pd.concat([dataframe,pd.get_dummies(dataframe.hour,prefix='hour')],axis=1)
    #region
    dataframe = pd.concat([dataframe,pd.get_dummies(dataframe.region,prefix='region')],axis=1)
    #adexchage
    dataframe = pd.concat([dataframe,pd.get_dummies(dataframe.adexchange,prefix='adexchange')],axis=1)
    #advertiser
    dataframe = pd.concat([dataframe,pd.get_dummies(dataframe.advertiser,prefix='advertiser')],axis=1)
    # slot width,hight,visibility,format
    dataframe = pd.concat([dataframe,pd.get_dummies(dataframe.slotwidth,prefix='slotwidth')],axis=1)
    dataframe = pd.concat([dataframe,pd.get_dummies(dataframe.slotheight,prefix='slotheight')],axis=1)
    dataframe = pd.concat([dataframe,pd.get_dummies(dataframe.slotvisibility,prefix='slotvisibility')],axis=1)    
    dataframe = pd.concat([dataframe,pd.get_dummies(dataframe.slotformat,prefix='slotformat')],axis=1)
    
    dataframe = dataframe.drop('weekday',axis=1)
    dataframe = dataframe.drop('hour',axis=1)
    dataframe = dataframe.drop('region',axis=1)
    dataframe = dataframe.drop('adexchange',axis=1)
    dataframe = dataframe.drop('slotwidth',axis=1)
    dataframe = dataframe.drop('slotheight',axis=1)
    dataframe = dataframe.drop('advertiser',axis=1)
    dataframe = dataframe.drop('slotvisibility',axis=1)
    dataframe = dataframe.drop('slotformat',axis=1)
    return dataframe

def encode_os_browser(dataframe):
    df_temp = pd.DataFrame(dataframe.useragent.str.split('_',1).tolist(), columns = ['OS','browser'])
    dataframe = pd.concat([dataframe,df_temp],axis=1)
    dataframe = dataframe.drop('useragent',axis=1)
    dataframe = pd.concat([dataframe,pd.get_dummies(dataframe.OS,prefix='OS')],axis=1)
    dataframe = dataframe.drop('OS',axis=1)
    dataframe = pd.concat([dataframe,pd.get_dummies(dataframe.browser,prefix='browser')],axis=1)
    dataframe = dataframe.drop('browser',axis=1)
    return dataframe
# 12. Encode slotprice into 5 ranges
def encode_slotprice(dataframe):
    slotprice_range = pd.DataFrame()
    slotprice_range['slotprices'] = pd.cut(dataframe.slotprice.values,5, labels=[1,2,3,4,5])
    dataframe = pd.concat([dataframe,slotprice_range],axis=1)
    dataframe = pd.concat([dataframe,pd.get_dummies(dataframe.slotprices,prefix='slotprice')],axis=1)
    dataframe = dataframe.drop('slotprice',axis=1)
    dataframe = dataframe.drop('slotprices',axis=1)
    return dataframe
def encode_usertags(dataframe):
    usertags = list(dataframe.usertag)
    unique_users = set()
    list_users = []
    for user in usertags:
        u = user.split(',')
        list_users.append(u)
        for us in u:
            unique_users.add(us)
    users = pd.DataFrame()
    for user in unique_users:
        users["user_"+user] = 0
    dataframe = pd.concat([dataframe,users],axis=1)
    for user in unique_users:
        datas = []
        for users in list_users:
            if user in users:
                datas.append(1)
            else:
                datas.append(0)
        dataframe["user_"+user] = datas
    dataframe = dataframe.drop('usertag',axis=1)
    return dataframe

In [7]:
xtrain = pip(train_x)
xtrain = encode_os_browser(xtrain)
xtrain = encode_usertags(xtrain)

In [8]:
xval = pip(val_x)
xval = encode_os_browser(xval)
xval = encode_usertags(xval)

In [14]:
x_train = xtrain
y_train = ytrain
x_val = xval
y_val =yval

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.datasets import imdb

C:\Users\USER\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### set parameters:
max_features = 5000
maxlen = 400
batch_size = 32
embedding_dims = 50
filters = 250
kernel_size = 3
hidden_dims = 250
epochs = 1

print(len(x_train), 'train sequences')
print(len(x_val), 'test sequences')


In [16]:
print('starting building the model:')

# define baseline model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(12, input_dim=219, activation='relu'))
    model.add(Dense(24, kernel_initializer='normal', activation='softmax'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model = baseline_model()
# Fit the model
model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=20, batch_size=200, verbose=2)
# Final evaluation of the model
scores = model.evaluate(x_val, y_val, verbose=0)
print("Baseline Error: %.2f%%" % (100-scores[1]*100))


starting building the model:
Train on 2430981 samples, validate on 303925 samples
Epoch 1/20
 - 76s - loss: 0.0383 - acc: 0.9978 - val_loss: 0.0055 - val_acc: 0.9993
Epoch 2/20
 - 73s - loss: 0.0056 - acc: 0.9993 - val_loss: 0.0049 - val_acc: 0.9993
Epoch 3/20
 - 70s - loss: 0.0049 - acc: 0.9993 - val_loss: 0.0048 - val_acc: 0.9993
Epoch 4/20
 - 70s - loss: 0.0047 - acc: 0.9993 - val_loss: 0.0047 - val_acc: 0.9993
Epoch 5/20
 - 70s - loss: 0.0046 - acc: 0.9993 - val_loss: 0.0047 - val_acc: 0.9993
Epoch 6/20
 - 71s - loss: 0.0046 - acc: 0.9993 - val_loss: 0.0047 - val_acc: 0.9993
Epoch 7/20
 - 70s - loss: 0.0045 - acc: 0.9993 - val_loss: 0.0047 - val_acc: 0.9993
Epoch 8/20
 - 70s - loss: 0.0045 - acc: 0.9993 - val_loss: 0.0047 - val_acc: 0.9993
Epoch 9/20
 - 70s - loss: 0.0044 - acc: 0.9993 - val_loss: 0.0047 - val_acc: 0.9994
Epoch 10/20
 - 70s - loss: 0.0044 - acc: 0.9993 - val_loss: 0.0048 - val_acc: 0.9993
Epoch 11/20
 - 70s - loss: 0.0044 - acc: 0.9993 - val_loss: 0.0047 - val_acc:

In [17]:
val_pred = model.predict(x_val)

In [18]:
val_pred

array([[0.00021644],
       [0.00028987],
       [0.00015264],
       ...,
       [0.00082504],
       [0.00031247],
       [0.00051086]], dtype=float32)

# tune the base_bid for linear bidding

In [25]:
avgCTR = 1785/2427741
best_base_bid = -1
best_metrics = [-1, -1, float('inf'), float('inf'), float('inf')]
new_val = df_val.copy()
f = open('linear_bid_cnn.txt','w')
basicbid = [71,72,73,74,75]

for base_bid in basicbid:
    bidprices = [x * base_bid / avgCTR for x in val_pred]
    new_val['bidprice'] = bidprices
    budget = 6250
    suc_bids = new_val.query('bidprice > payprice ')
    cost = 0
    clicks = 0
    imps = 0
    for index, row in suc_bids.iterrows():
        if cost <= budget:
            cost += row['payprice'] / 1000
            clicks += row['click']
            imps += 1
    eCPC = cost / clicks if clicks > 0 else float('inf')
    metrics_list = [clicks, clicks / imps * 100, cost, cost / imps, eCPC]
    f.write('current base_bid: ' + str(base_bid) + '\n')
    f.write('current metrics: ' + str(metrics_list) + '\n')    
    f.flush()
f.close()

In [ ]:
# # when basic_bid = 73, maxium click 140, CTR= 0.1091, 
# cost = 6205.830,avgCPM=0.048383, eCPC = 44.327357

# tune the base_bid for quadratic bidding

In [28]:
avgCTR = 1785/2427741
best_base_bid = -1
best_metrics = [-1, -1, float('inf'), float('inf'), float('inf')]
new_val = df_val.copy()
f = open('quadratic_bid_cnn.txt','w')
basicbid = [65,70,71,72,73,74,75]

for base_bid in basicbid:
    bidprices = [(x * base_bid / avgCTR +x*x*base_bid/avgCTR)for x in val_pred]
    new_val['bidprice'] = bidprices
    budget = 6250
    suc_bids = new_val.query('bidprice > payprice ')
    cost = 0
    clicks = 0
    imps = 0
    for index, row in suc_bids.iterrows():
        if cost <= budget:
            cost += row['payprice'] / 1000
            clicks += row['click']
            imps += 1
    eCPC = cost / clicks if clicks > 0 else float('inf')
    metrics_list = [clicks, clicks / imps * 100, cost, cost / imps, eCPC]
    f.write('current base_bid: ' + str(base_bid) + '\n')
    f.write('current metrics: ' + str(metrics_list) + '\n')     
    f.flush()
f.close()

In [29]:
# when basic_bid = 73, maxium click 140, CTR=0.1090, 
# cost = 6213.286,avgCPM=0.04841, eCPC = 44.3806

# tune the parameter c for ORTB 

In [46]:
best_base_bid = -1
best_metrics = [-1, -1, float('inf'), float('inf'), float('inf')]
new_val = df_val.copy()
f = open('ORBT_cnn.txt','w')

c_list = [9.5,9.8,10,10.1,10.2,10.5]
l = 1e-06

for c in c_list:
    bidprices = [(np.sqrt(((c/l)*x)+(c*c))-c)for x in val_pred]
    new_val['bidprice'] = bidprices
    budget = 6250
    suc_bids = new_val.query('bidprice > payprice ')
    cost = 0
    clicks = 0
    imps = 0
    for index, row in suc_bids.iterrows():
        if cost <= budget:
            cost += row['payprice'] / 1000
            clicks += row['click']
            imps += 1
    eCPC = cost / clicks if clicks > 0 else float('inf')
    metrics_list = [clicks, clicks / imps * 100, cost, cost / imps, eCPC]
    f.write('current c: ' + str(c) + '\n')
    f.write('current metrics: ' + str(metrics_list) + '\n')    
    f.flush()
f.close()

In [ ]:
# when c =10, maxium click 128, CTR=0.08728, 
# cost = 6086.725,avgCPM=0.04150, eCPC = 47.5525